In [1]:
from housing.entity.artifact_entity import DataIngestionArtifact
import pandas as pd 
data_ingestion_artifact= DataIngestionArtifact

In [2]:
train_file_path = r"D:\ML_project\Machine_Learning_Project_1\housing\artifact\data_ingestion\2023-10-28-18-58-22\ingested_data\train\housing.csv"

In [3]:
train_file_path = r"D:\ML_project\Machine_Learning_Project_1\housing\artifact\data_ingestion\2023-10-28-18-58-22\ingested_data\train\housing.csv"
df = pd.read_csv(train_file_path)

In [4]:
df.shape

(16512, 10)

In [5]:
df.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,INLAND
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,NEAR OCEAN
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,INLAND


In [6]:
x,y = df.drop(columns=['median_house_value'],axis=1), df[['median_house_value']]

In [7]:
x.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        158
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

In [8]:
from sklearn.impute import SimpleImputer

In [9]:
simple_imputer = SimpleImputer(strategy="median")

In [10]:
numerical_column = x.drop(columns=['ocean_proximity'],axis=1)

In [11]:
simple_imputer.fit_transform(numerical_column)

array([[-1.2146e+02,  3.8520e+01,  2.9000e+01, ...,  2.2370e+03,
         7.0600e+02,  2.1736e+00],
       [-1.1723e+02,  3.3090e+01,  7.0000e+00, ...,  2.0150e+03,
         7.6800e+02,  6.3373e+00],
       [-1.1904e+02,  3.5370e+01,  4.4000e+01, ...,  6.6700e+02,
         3.0000e+02,  2.8750e+00],
       ...,
       [-1.2272e+02,  3.8440e+01,  4.8000e+01, ...,  4.5800e+02,
         1.7200e+02,  3.1797e+00],
       [-1.2270e+02,  3.8310e+01,  1.4000e+01, ...,  1.2080e+03,
         5.0100e+02,  4.1964e+00],
       [-1.2214e+02,  3.9970e+01,  2.7000e+01, ...,  6.2500e+02,
         1.9700e+02,  3.1319e+00]])

In [12]:
df = simple_imputer.transform(numerical_column)

In [13]:
simple_imputer.feature_names_in_

array(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype=object)

In [14]:
# Median values for every column
simple_imputer.statistics_

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

In [15]:
#simple_imputer.transform(new_df)

In [16]:
x.ocean_proximity.value_counts()

<1H OCEAN     7277
INLAND        5262
NEAR OCEAN    2124
NEAR BAY      1847
ISLAND           2
Name: ocean_proximity, dtype: int64

In [17]:
simple_imputer = SimpleImputer(strategy="most_frequent")

In [18]:
#custom transformer

In [19]:
class FeatureGenerator():
    def __init__(self, strategy = "median"):
        self.strategy=strategy
    
    def fit(self,X):
        self.features = X.columns
        self.statistic_ = []
        for column in X.columns:
            self.statistic_.append(X[column].median())
            
    
    def trasform(self,X:pd.DataFrame):
        for idx,column in enumerate(X.columns):
            X[column].fillna(self.statistic_[idx])
        return X 
    
    def fit_transform(self,X):
        self.fit(X)
        return self.trasform(X)

In [20]:
for idx,column in enumerate(x.columns):
    print(idx,column)

0 longitude
1 latitude
2 housing_median_age
3 total_rooms
4 total_bedrooms
5 population
6 households
7 median_income
8 ocean_proximity


In [21]:
from sklearn.base import BaseEstimator ,TransformerMixin

In [22]:
# using transformer mixin - implemention fit_transorm 

class FeatureGenerator(BaseEstimator,TransformerMixin):
    
    def __init__(self, strategy = "median"):
        self.strategy=strategy
    
    def fit(self,X):
        self.features = X.columns
        self.statistic_ = []
        for column in X.columns:
            self.statistic_.append(X[column].median())
            
    
    def trasform(self,X:pd.DataFrame):
        for idx,column in enumerate(X.columns):
            X[column].fillna(self.statistic_[idx])
        return X 
    
    
    

In [23]:
class FeatureGenerator(BaseEstimator, TransformerMixin):

    def __init__(self, add_bedrooms_per_room=True,
                 total_rooms_ix=3,
                 population_ix=5,
                 households_ix=6,
                 total_bedrooms_ix=4, columns=None):
        """
        FeatureGenerator Initialization
        add_bedrooms_per_room: bool
        total_rooms_ix: int index number of total rooms columns
        population_ix: int index number of total population columns
        households_ix: int index number of  households columns
        total_bedrooms_ix: int index number of bedrooms columns
        """
        try:
            self.columns = columns
            if self.columns is not None:
                total_rooms_ix = self.columns.index(COLUMN_TOTAL_ROOMS)
                population_ix = self.columns.index(COLUMN_POPULATION)
                households_ix = self.columns.index(COLUMN_HOUSEHOLDS)
                total_bedrooms_ix = self.columns.index(COLUMN_TOTAL_BEDROOM)

            self.add_bedrooms_per_room = add_bedrooms_per_room
            self.total_rooms_ix = total_rooms_ix
            self.population_ix = population_ix
            self.households_ix = households_ix
            self.total_bedrooms_ix = total_bedrooms_ix
        except Exception as e:
            raise HousingException(e, sys) from e

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        try:
            room_per_household = X[:, self.total_rooms_ix] / \
                                 X[:, self.households_ix]
            population_per_household = X[:, self.population_ix] / \
                                       X[:, self.households_ix]
            if self.add_bedrooms_per_room:
                bedrooms_per_room = X[:, self.total_bedrooms_ix] / \
                                    X[:, self.total_rooms_ix]
                generated_feature = np.c_[
                    X, room_per_household, population_per_household, bedrooms_per_room]
            else:
                generated_feature = np.c_[
                    X, room_per_household, population_per_household]

            return generated_feature
        except Exception as e:
            raise HousingException(e, sys) from e



In [24]:
import numpy as np
arr1 = np.arange(10)
arr2 = np.arange(10)

In [25]:
np.c_[arr1,arr2]

array([[0, 0],
       [1, 1],
       [2, 2],
       [3, 3],
       [4, 4],
       [5, 5],
       [6, 6],
       [7, 7],
       [8, 8],
       [9, 9]])

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [27]:
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="median")),
    ('FeatureGenerator', FeatureGenerator),
    ('scaling', StandardScaler())
])

In [28]:
from sklearn.preprocessing import OneHotEncoder

In [29]:
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('OneHotEncoder', OneHotEncoder()),
    ('scaling', StandardScaler(with_mean=False))
])

In [30]:
num_column_name = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income']

cat_column_name = ['ocean_proximity']

In [31]:
preprocessing= ColumnTransformer([
    ('num_pipeline', num_pipeline,num_column_name ),
    ('cat_pipeline', cat_pipeline,cat_column_name )]
)

In [32]:
x.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity'],
      dtype='object')

In [33]:
train_file_path = r"D:\ML_project\Machine_Learning_Project_1\housing\artifact\data_ingestion\2023-10-28-18-58-22\ingested_data\train\housing.csv"
df = pd.read_csv(train_file_path)

In [34]:
#preprocessing.fit_transform(new_df)

In [46]:
new_df = pd.read_csv(r"D:\ML_project\Machine_Learning_Project_1\housing\artifact\data_ingestion\2023-10-28-18-58-22\ingested_data\test\housing.csv")

In [47]:
new_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-118.39,34.12,29.0,6447.0,1012.0,2184.0,960.0,8.2816,500001.0,<1H OCEAN
1,-120.42,34.89,24.0,2020.0,307.0,855.0,283.0,5.0099,162500.0,<1H OCEAN
2,-118.45,34.25,36.0,1453.0,270.0,808.0,275.0,4.3839,204600.0,<1H OCEAN
3,-118.10,33.91,35.0,1653.0,325.0,1072.0,301.0,3.2708,159700.0,<1H OCEAN
4,-117.07,32.77,38.0,3779.0,614.0,1495.0,614.0,4.3529,184000.0,NEAR OCEAN


In [53]:
#train_df = preprocessing.fit_transform(new_df)

In [39]:
import dill

In [49]:
with open('preprocessing.pkl', 'wb') as preprocessing_file:
    dill.dump(preprocessing,preprocessing_file )

In [50]:
from sklearn.linear_model import LinearRegression

In [ ]:
x = train_df
y = np.array(df.median_house_value)

In [57]:
#lr = LinearRegression(x,y)

In [54]:
lr.fit_intercept

True

In [58]:
import matplotlib.pyplot as plt

In [ ]:
y_hat = lr.predict(x)

In [ ]:
plt.plot(y,y_hat)

In [59]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y,y_hat)